# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model
- Compare and contrast model validation strategies

## Let's Get Started

We included the code to pre-process the Ames Housing dataset below. This is done for the sake of expediency, although it may result in data leakage and therefore overly optimistic model metrics.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

## Train-Test Split

Perform a train-test split with a test set of 20% and a random state of 4.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split, cross_val_score


In [3]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

### Fit a Model

Fit a linear regression model on the training set

In [4]:
# Import LinearRegression from sklearn.linear_model
from sklearn.linear_model import LinearRegression

In [5]:
# Instantiate and fit a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

### Calculate MSE

Calculate the mean squared error on the test set

In [6]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error


In [7]:
# Calculate MSE on test set
y_pred = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {mse_test:.4f}')

Test MSE: 0.1523


## Cross-Validation using Scikit-Learn

Now let's compare that single test MSE to a cross-validated test MSE.

In [8]:
# Import cross_val_score from sklearn.model_selection


In [9]:
# Find MSE scores for a 5-fold cross-validation
mse_cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
mse_cv_scores = -mse_cv_scores  # Convert negative MSE to positive

In [10]:
# Get the average MSE score
mse_cv_mean = mse_cv_scores.mean()
print(f'Cross-Validated MSE: {mse_cv_mean:.4f}')

Cross-Validated MSE: 0.1770


Compare and contrast the results. What is the difference between the train-test split and cross-validation results? Do you "trust" one more than the other?

In [11]:
# Your answer here


## Level Up: Let's Build It from Scratch!

### Create a Cross-Validation Function

Write a function `kfolds(data, k)` that splits a dataset into `k` evenly sized pieces. If the full dataset is not divisible by `k`, make the first few folds one larger then later ones.

For example, if you had this dataset:

In [12]:
example_data = pd.DataFrame({
    "color": ["red", "orange", "yellow", "green", "blue", "indigo", "violet"]
})
example_data

,color
0,red
1,orange
2,yellow
3,green
4,blue
5,indigo
6,violet


`kfolds(example_data, 3)` should return:

* a dataframe with `red`, `orange`, `yellow`
* a dataframe with `green`, `blue`
* a dataframe with `indigo`, `violet`

Because the example dataframe has 7 records, which is not evenly divisible by 3, so the "leftover" 1 record extends the length of the first dataframe.

In [15]:
def kfolds(data, k):
    n = len(data)
    fold_sizes = [n // k + (1 if i < n % k else 0) for i in range(k)]
    current_idx = 0
    folds = []
    for size in fold_sizes:
        folds.append(data.iloc[current_idx:current_idx + size])
        current_idx += size
    return folds

In [16]:
results = kfolds(example_data, 3)
for result in results:
    print(result, "\n")

    color
0     red
1  orange
2  yellow 

   color
3  green
4   blue 

    color
5  indigo
6  violet 



### Apply Your Function to the Ames Housing Data

Get folds for both `X` and `y`.

In [17]:
# Apply kfolds() to ames_data with 5 folds
X_folds = kfolds(X, 5)
y_folds = kfolds(y, 5)

### Perform a Linear Regression for Each Fold and Calculate the Test Error

Remember that for each fold you will need to concatenate all but one of the folds to represent the training data, while the one remaining fold represents the test data.

In [19]:
# Replace None with appropriate code
test_errs = []
k = 5

for n in range(k):
    # Split into train and test for the fold
    X_train = pd.concat([X_folds[j] for j in range(5) if j != n])
    X_test = X_folds[n]
    y_train = pd.concat([y_folds[j] for j in range(5) if j != n])
    y_test = y_folds[n]
    
    # Fit a linear regression model
    print(f'Test Errors for each fold: {test_errs}')
    
    # Evaluate test errors
    print(f'Average Test Error: {np.mean(test_errs):.4f}')

print(test_errs)

Test Errors for each fold: []
Average Test Error: nan
Test Errors for each fold: []
Average Test Error: nan
Test Errors for each fold: []
Average Test Error: nan
Test Errors for each fold: []
Average Test Error: nan
Test Errors for each fold: []
Average Test Error: nan
[]


C:\Users\Willy\Downloads\Anaconda\envs\learn-env\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Willy\Downloads\Anaconda\envs\learn-env\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


If your code was written correctly, these should be the same errors as scikit-learn produced with `cross_val_score` (within rounding error). Test this out below:

In [20]:
# Compare your results with sklearn results
print(f'Sklearn Cross-Validated MSE: {mse_cv_mean:.4f}')

Sklearn Cross-Validated MSE: 0.1770


This was a bit of work! Hopefully you have a clearer understanding of the underlying logic for cross-validation if you attempted this exercise.

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are more robust than train-test split.